# Language Model

- Data: Jaychou Lyrics
- Data Process:  
    - Only use a part of this dataset
    - Build index dictionary
    - Transform data to indices

In [1]:
import torch
import random
import zipfile

import renyan_utils as ry

## Data

### Read

In [ ]:
# saved to renyan_utils.py
def load_data_jay_lyrics(clip_num = 10000):
    with zipfile.ZipFile('data/book_data/jaychou_lyrics.txt.zip') as zin:
        with zin.open("jaychou_lyrics.txt") as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[:clip_num]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

In [2]:
# different every time
corpus_indices, char_to_idx, idx_to_char, vocab_size = ry.load_data_jay_lyrics()

### Sampling-Random

In [31]:
def data_iter_random(corpus_indices, batch_size, num_steps, device = None):
    num_examples = (len(corpus_indices) - 1) // num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)
    print(example_indices)
    
    def _data(pos):
        return corpus_indices[pos:pos + num_steps]
    
    if device == None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for i in range(epoch_size):
        i = i*batch_size
        batch_indices = example_indices[i:i + batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps + 1) for j in batch_indices]
        yield torch.tensor(X, dtype = torch.float32, device = device), torch.tensor(Y, dtype = torch.float32, device = device)

In [32]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size = 2, num_steps = 6):
    pass
#     print('X: ', X, '\nY: ', Y, '\n')

[2, 1, 3, 0]
